# ANKIT

# GUI Handwritten Digit Recognition

In [3]:
from tkinter import *
import cv2
import numpy as np
from PIL import ImageGrab
from tensorflow.keras.models import load_model

# Load the trained Keras model for digit recognition
model = load_model('mnist.h5')

# Define the image folder for saving screenshots
image_folder = "img/"

# Create the Tkinter window
root = Tk()
root.resizable(0, 0)
root.title("HDR")

# Initialize variables for tracking the drawing and image number
lastx, lasty = None, None
image_number = 0

# Create a canvas for drawing
cv = Canvas(root, width=640, height=480, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)

# Function to clear the canvas
def clear_widget():
    global cv
    cv.delete('all')

# Function to draw lines on the canvas
def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event.y
    cv.create_line((lastx, lasty, x, y), width=8, fill='black', capstyle=ROUND, smooth=TRUE, splinesteps=12)
    lastx, lasty = x, y

# Function to activate drawing on canvas
def activate_event(event):
    global lastx, lasty
    cv.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event.y

# Bind left mouse button press to activate drawing
cv.bind('<Button-1>', activate_event)

# Function to recognize and display the drawn digit
def Recognize_Digit():
    global image_number
    filename = f'img_{image_number}.png'
    widget = cv

    # Calculate the coordinates of the canvas area
    x = root.winfo_rootx() + widget.winfo_rootx()
    y = root.winfo_rooty() + widget.winfo_rooty()
    x1 = x + widget.winfo_width()
    y1 = y + widget.winfo_height()

    # Capture the canvas area as an image and save it
    ImageGrab.grab().crop((x, y, x1, y1)).save(image_folder + filename)

    # Load the saved image using OpenCV
    image = cv2.imread(image_folder + filename, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the thresholded image
    contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    # Iterate over detected contours
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)

        # Extract and preprocess the digit from the contour
        digit = th[y:y + h, x:x + w]
        resized_digit = cv2.resize(digit, (18, 18))
        padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)
        digit = padded_digit.reshape(1, 28, 28, 1)
        digit = digit / 255.0

        # Make a prediction using the model
        pred = model.predict([digit])[0]
        final_pred = np.argmax(pred)
        data = str(final_pred) + ' ' + str(int(max(pred) * 100)) + '%'

        # Draw prediction result on the image
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 1
        cv2.putText(image, data, (x, y - 5), font, fontScale, color, thickness)

    # Display the annotated image
    cv2.imshow('image', image)
    cv2.waitKey(0)

# Create a button to trigger digit recognition
btn_save = Button(text='Recognize Digit', command=Recognize_Digit)
btn_save.grid(row=2, column=0, pady=1, padx=1)

# Create a button to clear the canvas
button_clear = Button(text='Clear Widget', command=clear_widget)
button_clear.grid(row=2, column=1, pady=1, padx=1)

# Start the Tkinter main loop
root.mainloop()


1/1 [==============================] - 0s 15ms/step
